In [10]:
import pandas as pd

In [11]:
df = pd.read_parquet('data/profiles/devto')

In [12]:
def preprocess_meta_details(details: dict):
    details_data = details['details']
    data = {}

    features = ['Education', 'Location', 'Joined', 'Pronouns', 'Work']
    
    for f in features:
        if f in details_data:
            data[f.lower().replace(' ', '_')] = details_data[f]

    if data['joined'] != None:
        data['joined'] = data['joined'].replace('Joined on\xa0', '')

    urls = list(details['urls'])
    mail_idx = -1
    for i in range(len(urls)):
        if 'mailto:' in urls[i]:
            mail_idx = i

    if mail_idx != -1:
        data['email'] = urls[mail_idx].replace('mailto:', '')
        del urls[mail_idx]
    else:
        data['email'] = None

    data['websites'] = urls

    return data

In [13]:
def preprocess_qualifications(qualifications):
    features = ['Available for', 'Currently hacking on', 'Skills/Languages']
    skills = []
    for f in features:
        if f in qualifications:
            if qualifications[f] != None:
                data = [k.strip() for k in qualifications[f].split(",")]
                skills += data
    return {"skills": skills}

In [14]:
def preprocess(x):
    data = {}
    data['user_name'] = x['user_name']
    data['headline'] = x['headline']
    data.update(preprocess_meta_details(x['meta_details']))
    data.update(preprocess_qualifications(x["qualifications"]))
    data['date_added'] = x['date_added']
    return data    

In [15]:
preprocessed_data = df.apply(lambda x: preprocess(x), axis=1).to_list()

In [16]:
preprocessed_df = pd.DataFrame(preprocessed_data)

In [74]:
# preprocessed_df.to_parquet('data/preprocessed_profiles/devto', partition_cols=['date_added'])

In [17]:
preprocessed_df.head(3)

,user_name,headline,education,location,joined,pronouns,work,email,websites,skills,date_added
0,Vikas Chitturi,I am passionate about developing efficient ope...,None,Hyderabad,"Jan 17, 2019",None,Data Engineer at UnitedHealth Group,vikasch.1994@gmail.com,[https://github.com/absognety],"[Data science, Deep Learning, python, scala, J...",2024-03-10
1,Andrea Montes,I care about everything regards data. Data per...,Universidad Nacional de Colombia,"Bogotá, Colombia , South America","Jan 30, 2023",she/her,Data Engineer,mamontesp@gmail.com,[https://github.com/mamontesp],"[Dbt projects, ML for small data size + dbt + ...",2024-03-10
2,Rick Delpo,Rick Delpo - retired Senior Data Engineer from...,Georgetown University,"Connecticut, USA","Aug 1, 2020",None,Instructor at HowToLearnJava.com,rickdelpo@gmail.com,"[https://javasqlweb.org, https://github.com/ri...","[Java, JDBC, SQL, JQuery, React, AWS Lambda]",2024-03-10
